In [41]:
# Import Dependencies
# from census import Census
from config import census_api
import pandas as pd
import requests
import json
import os
import gmaps
import csv

In [42]:
# url for US Census data api call with codes for desired data fields

variables = ("B01003_001E,B02001_002E,B02001_003E,B02001_005E,B03001_003E,B02001_004E,B03002_003E,B03002_004E,B03002_005E,B03002_006E,B03002_007E,B03002_008E,B03002_009E,B03002_013E,B03002_014E,B03002_015E,B03002_016E,B03002_017E,B03002_018E,B03002_019E,B01002_001E,B01001_002E,B01001_026E,B19326_001E,B17001_002E,B27011_003E,B25105_001E,B08008_003E,B25033_008E,B25008_002E,B25008_001E,B25001_001E,B09002_001E,B19123_001E,B99051_002E,B99051_005E,B16010_002E,B16010_015E,B16010_028E,B16010_041E")

url = (f"https://api.census.gov/data/2020/acs/acs5?get={variables},NAME&for=place:44000&in=state:06&key={census_api}")

# Make api call and get json object of returned data 
response = requests.get(url)
response_json = response.json()

#Convert json object and make first row column names
census_df = pd.DataFrame(response_json)

census_df.columns = census_df.iloc[0]
census_df = census_df[1:]

census_df.head()

,B01003_001E,B02001_002E,B02001_003E,B02001_005E,B03001_003E,B02001_004E,B03002_003E,B03002_004E,B03002_005E,B03002_006E,...,B19123_001E,B99051_002E,B99051_005E,B16010_002E,B16010_015E,B16010_028E,B16010_041E,NAME,state,place
1,3973278,1944144,348701,468006,1909808,28732,1132606,334969,6037,460730,...,830131,2529680,1443598,601860,520611,658963,986144,"Los Angeles city, California",06,44000


In [43]:
# Re_labeled the columns.
census_df = census_df.rename({'B01003_001E':'total_pop','B02001_002E':'pop_white','B02001_003E':'pop_black','B02001_005E':'pop_asian','B03001_003E':'pop_hispanic','B02001_004E':'pop_native_amer','B03002_003E':'pop_not_hisp_white','B03002_004E':'pop_not_hisp_black','B03002_005E':'pop_not_hisp_na','B03002_006E':'pop_not_hisp_asian','B03002_007E':'pop_not_hisp_nh_pi','B03002_008E':'pop_not_hisp_other','B03002_009E':'pop_not_hisp_two_more','B03002_013E':'pop_hisp_white','B03002_014E':'B03002_014','B03002_015E':'pop_hisp_na','B03002_016E':'pop_hisp_asian','B03002_017E':'pop_hisp_nh_pi','B03002_018E':'pop_hisp_other','B03002_019E':'pop_hisp_two_more','B01002_001E':'med_age','B01001_002E':'pop_male','B01001_026E':'pop_female','B19326_001E':'med_income','B17001_002E':'pop_below_poverty','B27011_003E':'pop_employed','B25105_001E':'month_housing_costs','B08008_003E':'pop_work_out_res_area','B25033_008E':'pop_rented','B25008_002E':'pop_owned','B25008_001E':'pop_occupied','B25001_001E':'total_hu','B09002_001E':'pop_under_18','B19123_001E':'total_families','B99051_002E':'pop_native_born','B99051_005E':'pop_foreign_born','B16010_002E':'pop_educ_lt_hs','B16010_015E':'pop_educ_hs','B16010_028E':'pop_educ_some_col','B16010_041E':'pop_educ_mt_ba'
}, axis=1) 
census_df

,total_pop,pop_white,pop_black,pop_asian,pop_hispanic,pop_native_amer,pop_not_hisp_white,pop_not_hisp_black,pop_not_hisp_na,pop_not_hisp_asian,...,total_families,pop_native_born,pop_foreign_born,pop_educ_lt_hs,pop_educ_hs,pop_educ_some_col,pop_educ_mt_ba,NAME,state,place
1,3973278,1944144,348701,468006,1909808,28732,1132606,334969,6037,460730,...,830131,2529680,1443598,601860,520611,658963,986144,"Los Angeles city, California",06,44000


In [44]:
# Display all column_labels.
census_df.columns

Index(['total_pop', 'pop_white', 'pop_black', 'pop_asian', 'pop_hispanic',
       'pop_native_amer', 'pop_not_hisp_white', 'pop_not_hisp_black',
       'pop_not_hisp_na', 'pop_not_hisp_asian', 'pop_not_hisp_nh_pi',
       'pop_not_hisp_other', 'pop_not_hisp_two_more', 'pop_hisp_white',
       'B03002_014', 'pop_hisp_na', 'pop_hisp_asian', 'pop_hisp_nh_pi',
       'pop_hisp_other', 'pop_hisp_two_more', 'med_age', 'pop_male',
       'pop_female', 'med_income', 'pop_below_poverty', 'pop_employed',
       'month_housing_costs', 'pop_work_out_res_area', 'pop_rented',
       'pop_owned', 'pop_occupied', 'total_hu', 'pop_under_18',
       'total_families', 'pop_native_born', 'pop_foreign_born',
       'pop_educ_lt_hs', 'pop_educ_hs', 'pop_educ_some_col', 'pop_educ_mt_ba',
       'NAME', 'state', 'place'],
      dtype='object', name=0)

In [45]:
# Re_Order the columns. 
census_df=census_df[["total_pop",
'pop_male','pop_female','pop_white', 'pop_black', 'pop_asian', 'pop_hispanic',
'pop_native_amer', 'pop_not_hisp_white', 'pop_not_hisp_black',
'pop_not_hisp_na', 'pop_not_hisp_asian', 'pop_not_hisp_nh_pi',
'pop_not_hisp_other', 'pop_not_hisp_two_more', 'pop_hisp_white',
'B03002_014', 'pop_hisp_na', 'pop_hisp_asian', 'pop_hisp_nh_pi',
'pop_hisp_other', 'pop_hisp_two_more', 'med_age',
'med_income', 'pop_below_poverty', 'pop_employed',
'month_housing_costs', 'pop_work_out_res_area', 'pop_rented',
'pop_owned', 'pop_occupied', 'total_hu', 'pop_under_18',
'total_families', 'pop_native_born', 'pop_foreign_born',
'pop_educ_lt_hs', 'pop_educ_hs', 'pop_educ_some_col', 'pop_educ_mt_ba','NAME', 'state', 'place']]
census_df

,total_pop,pop_male,pop_female,pop_white,pop_black,pop_asian,pop_hispanic,pop_native_amer,pop_not_hisp_white,pop_not_hisp_black,...,total_families,pop_native_born,pop_foreign_born,pop_educ_lt_hs,pop_educ_hs,pop_educ_some_col,pop_educ_mt_ba,NAME,state,place
1,3973278,1966568,2006710,1944144,348701,468006,1909808,28732,1132606,334969,...,830131,2529680,1443598,601860,520611,658963,986144,"Los Angeles city, California",06,44000


In [46]:
# Show datatypes.
census_df.dtypes

0
total_pop                object
pop_male                 object
pop_female               object
pop_white                object
pop_black                object
pop_asian                object
pop_hispanic             object
pop_native_amer          object
pop_not_hisp_white       object
pop_not_hisp_black       object
pop_not_hisp_na          object
pop_not_hisp_asian       object
pop_not_hisp_nh_pi       object
pop_not_hisp_other       object
pop_not_hisp_two_more    object
pop_hisp_white           object
B03002_014               object
pop_hisp_na              object
pop_hisp_asian           object
pop_hisp_nh_pi           object
pop_hisp_other           object
pop_hisp_two_more        object
med_age                  object
med_income               object
pop_below_poverty        object
pop_employed             object
month_housing_costs      object
pop_work_out_res_area    object
pop_rented               object
pop_owned                object
pop_occupied             object
total_

In [47]:
census_df.to_csv("census_data_2020.csv", index=False)